In [37]:
import matplotlib.pyplot as plt
import numpy as np
from pycocotools.coco import COCO
import json
from pycocotools.cocoeval import COCOeval
import pickle

# pickleFilePath = '../data/data/data/da_D0.pkl'
pickleFilePath = '../data/data/ub_D0.pkl'
pickleFile = open(pickleFilePath,'rb')
pred_pkl = pickle.load(pickleFile)
anno_path = '../data/bdd100k/annotations/det_val_D0.json'
# save_path = '../data/experiment/result.json'
coco = COCO(anno_path)

def get_pred_data_pkl(img_id):
    index = img_id - 1
    cls_score = pred_pkl[index]['pred_instances']['scores'].numpy()
    bbox = pred_pkl[index]['pred_instances']['bboxes'].numpy()
    cat_ids = pred_pkl[index]['pred_instances']['labels'].numpy()
    return cat_ids,cls_score,bbox
def get_gt_data(img_id):
    anno_id = coco.getAnnIds(img_id)
    anno_data = coco.loadAnns(anno_id)
    bbox_list = []; cate_list = []
    for anno in anno_data:
        x1, y1, w, h = anno['bbox']
        bbox_list.append([x1, y1, x1+w, y1+h])
        cate_list.append(anno['category_id'])
    bbox_data = np.array(bbox_list)
    cate_data = np.array(cate_list)
    return cate_data,bbox_data

loading annotations into memory...
Done (t=1.00s)
creating index...
index created!


In [42]:
imgids = np.zeros(10)
scores = np.zeros(10)

for index in range(3067):
    imid = index +1
    xx,yy = get_gt_data(imid)
    for catid in xx:
        imgids[catid-1] += 1
        # imgids[catid]+=1

print(imgids)
print(np.argsort(-imgids))

for index in range(3067):
    imid = index + 1
    cat1,score1,bbox1 = get_pred_data_pkl(imid)
    # cat1 = cat1[np.where(score1>0)[0]]
    for catid in cat1:
        # scores[catid-1] += 1
        scores[catid] +=1
print(scores)

print(np.argsort(-scores))

#     indes = np.where(score1 > 0 )[0]
#     cat1 = cat1[indes]
#     for ic in [1,2,3,4,5,6,7,8,9,10]:
#         if (ic in cat1) & (imgids[ic-1]==0):
#             imgids[ic-1]=imid
#
# imgids

[0.0000e+00 3.9700e+02 3.4025e+04 8.7700e+02 2.0550e+03 0.0000e+00
 0.0000e+00 1.0740e+04 1.2313e+04 2.0000e+00]
[2 8 7 4 3 1 9 0 5 6]
[     0.   9155. 132543.  13304.  27555.      0.      0.  41897.  80765.
    461.]
[2 8 7 4 3 1 9 0 5 6]


In [45]:
coco_result = []
for sample_img in pred_pkl:
    result = []
    img_id = int(sample_img['img_id'])
    ann_num = len(sample_img['pred_instances']['labels'])
    for index in range(ann_num):
        label = int(sample_img['pred_instances']['labels'][index])
        score = float(sample_img['pred_instances']['scores'][index])
        bbox_xyxy = sample_img['pred_instances']['bboxes'][index].numpy().tolist()
        label +=1
        # label_map = []
        #
        # label = label_map[label-1]
        # 卡分
        if score < 0.05:
            continue
        xs = bbox_xyxy[0]
        ys = bbox_xyxy[1]
        ws = bbox_xyxy[2] - xs -1
        hs = bbox_xyxy[3] - ys -1
        bbox_xywh = [float(xs), float(ys), float(ws), float(hs)]   
        result += [
            {
                'image_id': img_id,
                'category_id': label,
                'bbox': bbox_xywh,
                'score': score
            }
        ]
    # result = sorted(result, key=lambda x: x['score'])[-100:]
    coco_result += result
coco_dt = coco.loadRes(coco_result)
coco_eval = COCOeval(coco, coco_dt,'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

Loading and preparing results...
DONE (t=0.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=30.71s).
Accumulating evaluation results...
DONE (t=2.70s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.332
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.562
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.324
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.392
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.652
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.201
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.444
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [26]:
img = 2000
ann = coco_dt.getAnnIds(img)
anndata = coco_dt.loadAnns(ann)
print(coco_dt.loadImgs(img))
ann_gt = coco.getAnnIds(img)
anndata_gt = coco.loadAnns(ann_gt)
print(coco.loadImgs(img))



[{'file_name': 'c1f65250-4bf4a042.jpg', 'height': 720, 'width': 1280, 'id': 2000}]
[{'file_name': 'c1f65250-4bf4a042.jpg', 'height': 720, 'width': 1280, 'id': 2000}]


In [27]:
xx1 = []
xx2 = []
for instance in anndata:
    xx1.append(instance['category_id'])
for instance in anndata_gt:
    xx2.append(instance['category_id'])
print(xx1)
print(xx2)

[2, 7, 8, 2, 7, 7, 2, 2, 2, 2, 2, 2, 8, 2, 7]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 9, 8, 3, 3, 3, 9]


In [28]:
from PIL import Image, ImageDraw
# 定义画布尺寸
canvas_width = 1280
canvas_height = 720

# 创建空白画布
canvas = Image.new('RGB', (canvas_width, canvas_height), 'white')

# 创建绘图对象
draw = ImageDraw.Draw(canvas)
for instance in anndata_gt:
    bbox = instance['bbox']
    print(instance['category_id'])
    colors = [
    (255, 0, 0),      # 红色
    (0, 255, 0),      # 绿色
    (0, 0, 255),      # 蓝色
    (255, 255, 0),    # 黄色
    (255, 0, 255),    # 品红色
    (0, 255, 255),    # 青色
    (128, 0, 0),      # 深红色
    (0, 128, 0),      # 深绿色
    (0, 0, 128),      # 深蓝色
    (128, 128, 128)   # 灰色
    ]
    # 定义边界框数据
    x = bbox[0]
    y = bbox[1]
    w = bbox[2]
    h = bbox[3]
    # 绘制边界框
    draw.rectangle([(x, y), (x+w, y+h)], outline=colors[instance['category_id']-1], width=2)
# 保存画布
canvas.save('./img/output_gt.png')

print('------------------')

canvas1 = Image.new('RGB', (canvas_width, canvas_height), 'white')
draw1 = ImageDraw.Draw(canvas1)
for instance in anndata:
    if instance['score'] < 0.05:
        continue
    bbox = instance['bbox']
    print(instance['category_id'])
    colors = [
    (255, 0, 0),      # 红色
    (0, 255, 0),      # 绿色
    (0, 0, 255),      # 蓝色
    (255, 255, 0),    # 黄色
    (255, 0, 255),    # 品红色
    (0, 255, 255),    # 青色
    (128, 0, 0),      # 深红色
    (0, 128, 0),      # 深绿色
    (0, 0, 128),      # 深蓝色
    (128, 128, 128)   # 灰色
    ]
    # 定义边界框数据
    x = bbox[0]
    y = bbox[1]
    w = bbox[2]
    h = bbox[3]
    # 绘制边界框
    draw1.rectangle([(x, y), (x+w, y+h)], outline=colors[instance['category_id']-1], width=2)
# 保存画布
canvas1.save('./img/output_pred.png')

3
3
3
3
3
3
3
3
3
8
8
8
9
8
3
3
3
9
------------------
2
7
8
2
7
7
2
2
2
2
2
2
8
2
7


In [29]:
from PIL import Image, ImageDraw
ann = coco_dt.getAnnIds(img)
anndata = coco_dt.loadAnns(ann)
ann_gt = coco.getAnnIds(img)
anndata_gt = coco.loadAnns(ann_gt)
imagepath = '../data/bdd100k/images/100k/val/'
imagepath = imagepath + coco.loadImgs(1)[0]['file_name']

canvas = Image.open(imagepath)
canvas1 = Image.open(imagepath)
draw = ImageDraw.Draw(canvas)

In [39]:
from PIL import Image, ImageDraw

CLASSES = [
    "pedestrian",
    "rider",
    "car",
    "truck",
    "bus",
    "train",
    "motorcycle",
    "bicycle",
    "traffic light",
    "traffic sign",
]

classes = ["person", "rider", "car", "bus", "truck", "bike",
    "motor", "traffic light", "traffic sign", "train"]

# imagepath = '../data/bdd100k/images/100k/val/'
# 打开图
for index in range(3067):
    id = index + 1
    print(id)
    ann = coco_dt.getAnnIds(id)
    anndata = coco_dt.loadAnns(ann)
    ann_gt = coco.getAnnIds(id)
    anndata_gt = coco.loadAnns(ann_gt)
    imagepath = '../data/bdd100k/images/100k/val/' + coco.loadImgs(id)[0]['file_name']
    canvas = Image.open(imagepath)
    canvas1 = Image.open(imagepath)
    draw = ImageDraw.Draw(canvas)
    for instance in anndata_gt:
        bbox = instance['bbox']
        colors = [
        (255, 0, 0),      # 红色
        (0, 255, 0),      # 绿色
        (0, 0, 255),      # 蓝色
        (255, 255, 0),    # 黄色
        (255, 0, 255),    # 品红色
        (0, 255, 255),    # 青色
        (128, 0, 0),      # 深红色
        (0, 128, 0),      # 深绿色
        (0, 0, 128),      # 深蓝色
        (128, 128, 128)   # 灰色
        ]
        # 定义边界框数据
        x = bbox[0]
        y = bbox[1]
        w = bbox[2]
        h = bbox[3]
        # 绘制边界框
        draw.rectangle([(x, y), (x+w, y+h)], outline=colors[instance['category_id']-1], width=2)
        label = instance['category_id']
        draw.text((x,y),str(label)+" "+classes[label-1])
        draw.text((x,y),str(instance['category_id']))
    # 保存画布
    canvas.save('./img/gt/'+str(id)+'.jpg')

    draw1 = ImageDraw.Draw(canvas1)
    for instance in anndata:
        if instance['score'] < 0.05:
            continue
        bbox = instance['bbox']
        colors = [
        (255, 0, 0),      # 红色
        (0, 255, 0),      # 绿色
        (0, 0, 255),      # 蓝色
        (255, 255, 0),    # 黄色
        (255, 0, 255),    # 品红色
        (0, 255, 255),    # 青色
        (128, 0, 0),      # 深红色
        (0, 128, 0),      # 深绿色
        (0, 0, 128),      # 深蓝色
        (128, 128, 128)   # 灰色
        ]
        # 定义边界框数据
        x = bbox[0]
        y = bbox[1]
        w = bbox[2]
        h = bbox[3]
        # 绘制边界框
        draw1.rectangle([(x, y), (x+w, y+h)], outline=colors[instance['category_id']-1], width=2)
        label = instance['category_id']
        draw1.text((x,y),str(label)+" "+classes[label-1])
    # 保存画布
    canvas1.save('./img/pred/'+str(id)+'.jpg')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131


KeyboardInterrupt: 

In [ ]:
def get_pred_data_pkl(img_id):
    index = img_id - 1
    cls_score = pred_pkl[index]['pred_instances']['scores'].numpy()
    bbox = pred_pkl[index]['pred_instances']['bboxes'].numpy()
    cat_ids = pred_pkl[index]['pred_instances']['labels'].numpy()
    name = pred_pkl[index]['img_path']
    return cat_ids,cls_score,bbox,name
def get_gt_data(img_id):
    anno_id = coco.getAnnIds(img_id)
    anno_data = coco.loadAnns(anno_id)
    name = coco.loadImgs(img_id)['file_name']
    bbox_list = []; cate_list = []
    for anno in anno_data:
        x1, y1, w, h = anno['bbox']
        bbox_list.append([x1, y1, x1+w, y1+h])
        cate_list.append(anno['category_id'])
    bbox_data = np.array(bbox_list)
    cate_data = np.array(cate_list)
    return cate_data,bbox_data

In [10]:
def calculate_iou(box1, box2):
    # Calculate Intersection over Union (IoU) between two bounding boxes
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area
    return iou

def calculate_ap(recall, precision):
    # Calculate Average Precision (AP) given recall and precision arrays
    recall = np.concatenate(([0.], recall, [1.]))
    precision = np.concatenate(([0.], precision, [0.]))

    for i in range(len(precision) - 1, 0, -1):
        precision[i - 1] = max(precision[i - 1], precision[i])

    indices = np.where(recall[1:] != recall[:-1])[0]
    ap = np.sum((recall[indices + 1] - recall[indices]) * precision[indices + 1])
    return ap

def calculate_map(img_id,iou_threshold=0.5):
    cat_id_gt, bbox_gt = get_gt_data(img_id)
    cat_id_pred,scores_pred,bbox__pred = get_pred_data_pkl(img_id)
    average_precisions = []
    for index in range(10):
        cid = index + 1
        index_class_gt = np.where(cat_id_gt == cid)[0]
        index_class_pred = np.where(cat_id_pred == cid)[0]
        num_pred = index_class_pred.shape[0]
        num_gt = index_class_gt.shape[0]

        if num_pred == 0:
            average_precisions.append(0)
            continue

        class_pred_score = scores_pred[index_class_pred]
        class_pred_bbox = bbox__pred[index_class_pred]
        class_pred_cid = cat_id_pred[index_class_pred]
        class_gt_bbox = bbox_gt[index_class_gt]

        class_sorted_indices = np.argsort(-class_pred_score)
        class_pred_score_sorted = class_pred_score[class_sorted_indices]
        class_pred_bbox_sorted = class_pred_bbox[class_sorted_indices]
        class_pred_cid_sorted = class_pred_cid[class_sorted_indices]

        true_positives = np.zeros(num_pred)
        false_positives = np.zeros(num_pred)
        matched_ground_truth = set()

        for index_in_class in range(num_pred):
            if num_gt == 0:
                false_positives[index_in_class] = 1
                continue

            ious = [calculate_iou(class_pred_bbox_sorted[index_in_class], box) for box in class_gt_bbox]
            max_iou = max(ious)
            max_iou_index = np.argmax(ious)

            if max_iou >= iou_threshold and max_iou_index not in matched_ground_truth:
                true_positives[index_in_class] = 1
                matched_ground_truth.add(max_iou_index)
            else:
                false_positives[index_in_class] = 1

        true_positives = np.cumsum(true_positives)
        false_positives = np.cumsum(false_positives)
        recall = true_positives / num_gt
        precision = true_positives / (true_positives + false_positives)

        ap = calculate_ap(recall, precision)
        average_precisions.append(ap)

    mAP = np.mean(average_precisions)
    return mAP